In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from numba import njit

![Alt text](image-1.png)

In [ ]:
df_fesx = pd.read_csv("./data/fesx_M15_2020-06-20_F.US.DSXM20.scid_BarData.txt", sep=", ", engine = "python")
df_fesx["Datetime"] = pd.to_datetime(df_fesx.Date + "-" + df_fesx.Time,format="%Y/%m/%d-%H:%M:%S")
df_fesx = df_fesx[['Datetime','Open', 'High', 'Low', 'Last', 'Volume']].rename(columns={"Last":"Close"})#.set_index("Datetime")
df_fesx = df_fesx.rename(columns={"Last":"Close"})
df_fesx.head()

In [ ]:
df_fesx['diff_high'] = df_fesx['High'].diff()
df_fesx['diff_low'] = df_fesx['Low'].diff()

In [ ]:
@njit()
def monotonic_segment(high, low, high_diff, low_diff):
    # threshhold for monotonicity
    th = 3
    # group monotonic segments
    g = 1
    state = 0 # 0: flat, 1: increasing, -1: decreasing
    peak = 0
    counter = 0
    q = int(high.shape[0]/10)
    segs = np.zeros((3,q))
    seg_idx = 0
    wave_started  = False
    for idx in range(high.shape[0]):
        #high_value = high[idx]
        #low_value = low[idx]
        high_diff_value = high_diff[idx]
        low_diff_value = low_diff[idx]

        if high_diff_value > 0:
            if state != 1:
                if (state == -1) and (counter > th):
                    segs[0,seg_idx] = -1
                    segs[1,seg_idx] = idx - counter
                    segs[2,seg_idx] = idx - 1
                    seg_idx += 1
                state = 1
                counter = 1
            else:
                counter += 1
        elif low_diff_value < 0:
            if state != -1:
                if (state == 1) and (counter > th):
                    segs[0,seg_idx] = 1
                    segs[1,seg_idx] = idx - counter
                    segs[2,seg_idx] = idx - 1
                    seg_idx += 1
                state = -1
                counter = 1
            else:
                counter += 1
    return segs[:,:seg_idx]



segs = monotonic_segment(df_fesx.High.to_numpy(), 
                df_fesx.High.to_numpy(),
                df_fesx.diff_high.to_numpy(),
                df_fesx.diff_low.to_numpy()      
                  )


dict_wave_type = {1:"High", -1:"Low"}

fig = go.Figure(data= [
    go.Scatter(x=df_fesx.index, y=df_fesx.High,name = 'High',mode="lines+markers", marker_color = "green"),
    go.Scatter(x=df_fesx.index, y=df_fesx.Low,name = 'Low',mode="lines+markers", marker_color = "red"),
    
    go.Candlestick(x=df_fesx.index,
                open=df_fesx['Open'],
                high=df_fesx['High'],
                low=df_fesx['Low'],
                close=df_fesx['Close'],
                name = 'Candlestick'
                )
    ]
    )

fig.update_layout(
    xaxis_rangeslider_visible= False
)


for seg_idx in range(segs.shape[1]):
    start_idx = int(segs[1,seg_idx])
    end_idx = int(segs[2,seg_idx] )
    col_name = dict_wave_type[int(segs[0,seg_idx])]
    start_wave = df_fesx.loc[start_idx, col_name]
    end_wave = df_fesx.loc[end_idx, col_name]
    df_fesx.High.loc[0]
    fig.add_trace(
      go.Scatter(x=[start_idx, end_idx], y=[start_wave, end_wave],name = 'level',mode="lines", marker_color = "pink", showlegend=False)
   )
    #print(start_idx, end_idx, start_wave, end_wave)
    continue

fig.update_layout(height=250, width=900,
     margin=dict(l=0, r=0, t=0, b=0),             
                  )
fig.show()